In [115]:
import pandas as pd
import numpy as np
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns

In [2]:
#Make a big data set with 
order_products_prior = pd.read_csv("order_products__prior.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")

In [3]:
orders=orders.loc[orders['eval_set']=='prior']

In [68]:
#We will work with 5000 users, last order will be in test set
users=range(1,5001)
orders_set=orders[orders["user_id"].isin(users)]
idx_test = orders_set.groupby(['user_id'])['order_number'].transform(max) == orders_set['order_number']
orders_set_test=orders_set[idx_test]

orders_set_testlist=np.unique(orders_set_test.order_id)
orders_set_train = orders_set[-orders_set["order_id"].isin(orders_set_testlist)]
orders_set_trainlist=np.unique(orders_set_train.order_id)

train= order_products_prior[order_products_prior["order_id"].isin(orders_set_trainlist)]
test=order_products_prior[order_products_prior["order_id"].isin(orders_set_testlist)]
train = pd.merge(train, orders_set_train, on='order_id', how='left')
test=pd.merge(test, orders_set_test, on='order_id', how='left')

In [5]:
print('Train set has %f orders',len(np.unique(train.order_id)))
print('Test set has %f orders',len(np.unique(test.order_id)))
print('Train set has %f users',len(np.unique(train.user_id)))
print('Test set has %f users',len(np.unique(test.user_id)))

Train set has %f orders 71832
Test set has %f orders 5000
Train set has %f users 5000
Test set has %f users 5000


In [6]:
products = pd.read_csv("products.csv")
train=train.merge(products, on='product_id')

In [7]:
def f1(y_true,y_pred):    
    y_true = set(y_true)
    y_pred = set(y_pred)
    cross_size = len(y_true & y_pred)
    if cross_size == 0: return 0.
    p = 1. * cross_size / len(y_pred)
    r = 1. * cross_size / len(y_true)
    return 2 * p * r / (p + r)

In [47]:
#Mean Average Precision (MAP)
def apk(y_true,y_pred, k=10):
    """
    AP rewards you for giving correct recommendations,
    AP rewards you for front-loading the recommendations that are most likely to be correct,
    AP will never penalize you for adding additional recommendations to your list — just make sure you front-load the best ones.
    """
    if len(y_pred)>k:
        y_pred = y_pred[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(y_pred):
        if p in y_true and p not in y_pred[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not y_true:
        return 0.0

    return score / min(len(y_true), k)




In [ ]:
#AUC

In [50]:
#Recommend top10 most sale items from train set
top_train=train["product_id"].value_counts().head(10)
test_history=test.groupby('order_id').aggregate({'product_id':lambda x: list(x)})
top_train=np.array(top_train.values)

test_history['f1']=0
test_history['apk']=0
for i in test_history.index:
    test_history['f1'][i]=f1(test_history['product_id'][i],top_train)
    test_history['apk'][i]=apk(test_history['product_id'][i],top_train)
    

print('f1 mean:',np.mean(test_history.f1))
print('apk mean:',np.mean(test_history.apk))

/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


f1 mean: 0.0
apk mean: 0.0


In [51]:
#Recommend top10 most reorder items from train set
top_reorder_train=train.groupby("product_id")["reordered"].aggregate({'Total_reorders': 'sum'})['Total_reorders'].sort_values(ascending=False).head(10)
top_reorder_train=np.array(top_reorder_train.values)
test_history=test.groupby('order_id').aggregate({'product_id':lambda x: list(x)})

test_history['apk']=0
test_history['f1']=0
for i in test_history.index:
    test_history['f1'][i]=f1(test_history['product_id'][i],top_reorder_train)
    test_history['apk'][i]=apk(test_history['product_id'][i],top_train)

print('f1 mean:',np.mean(test_history.f1))
print('apk mean:',np.mean(test_history.apk))

/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


f1 mean: 0.0
apk mean: 0.0


In [72]:
#Recommend last order of the user
last_orders = train.groupby("user_id")["order_number"].aggregate(np.max)
t = pd.merge(left=last_orders.reset_index(),right=train,how='inner',on=['user_id', 'order_number'])
t_last_order=t.groupby('order_id').aggregate({'product_id':lambda x: list(x)})
t_last_order = pd.merge(t_last_order,train[['order_id','user_id']],on='order_id')
t_last_order=t_last_order.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

test_history=test.groupby('order_id').aggregate({'product_id':lambda x: list(x)})
test_history = pd.merge(test_history,test[['order_id','user_id']],on='order_id')
test_history=test_history.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

t_last_order = pd.merge(t_last_order,test_history,on='user_id')
t_last_order=t_last_order.sort_values('user_id')
t_last_order.head()

,order_id_x,product_id_x,user_id,order_id_y,product_id_y
3375,2295261,"[49235, 46149, 25133, 196, 10258, 12427]",1,2550362,"[196, 46149, 39657, 38928, 25133, 10258, 35951..."
4800,3268552,"[42342, 30908, 79, 5869, 44303, 16521, 39877, ...",2,839880,"[24852, 16589, 1559, 19156, 18523, 22825, 2741..."
765,521107,"[39190, 47766, 21903, 43961, 17668]",3,1402502,"[39190, 18599, 23650, 21903, 47766, 24810]"
154,94891,"[22199, 25146]",4,2557754,"[26576, 25623, 21573]"
3338,2267326,"[26604, 11777, 20754, 24231, 40706]",5,157374,"[27344, 24535, 43693, 40706, 16168, 21413, 139..."


In [56]:
t_last_order['f1']=0
t_last_order['apk']=0
for i in t_last_order.index:
    t_last_order['f1'][i]=f1(t_last_order['product_id_y'][i],t_last_order['product_id_x'][i])
    t_last_order['apk'][i]=apk(t_last_order['product_id_y'][i],t_last_order['product_id_x'][i])

print('f1 mean:',np.mean(t_last_order.f1))
print('apk mean:',np.mean(test_history.apk))

/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


f1 mean: 0.01893439013650374
apk mean: 0.0


In [111]:
#Recommend just reorder articles during last order
last_orders = train.groupby("user_id")["order_number"].aggregate(np.max)
t = pd.merge(left=last_orders.reset_index(), right=train[train.reordered == 1], how='inner',
                 on=['user_id', 'order_number'])
t_last_order = t.groupby('order_id').aggregate({'product_id': lambda x: list(x)})
t_last_order = pd.merge(t_last_order, train[['order_id', 'user_id']], on='order_id')
t_last_order = t_last_order.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

test_history = test.groupby('order_id').aggregate({'product_id': lambda x: list(x)})
test_history = pd.merge(test_history, test[['order_id', 'user_id']], on='order_id')
test_history = test_history.drop_duplicates(subset=['order_id', 'user_id'], keep='first')

t_last_order = pd.merge(t_last_order, test_history, on='user_id')
t_last_order = t_last_order.sort_values('user_id')
t_last_order.head()

,order_id_x,product_id_x,user_id,order_id_y,product_id_y
3049,2295261,"[49235, 46149, 25133, 196, 10258, 12427]",1,2550362,"[196, 46149, 39657, 38928, 25133, 10258, 35951..."
699,521107,"[39190, 47766, 21903, 43961, 17668]",3,1402502,"[39190, 18599, 23650, 21903, 47766, 24810]"
3017,2267326,"[26604, 11777]",5,157374,"[27344, 24535, 43693, 40706, 16168, 21413, 139..."
414,298250,"[38293, 21903]",6,998866,"[49401, 25659, 8424]"
513,369015,"[47272, 37999, 21137, 12196, 23712]",7,2452257,"[47272, 29993, 31683, 27690, 9598, 13198, 3039..."


In [116]:
test_history

,order_id,product_id,user_id
0,40,"[10070, 42450, 33198, 34866]",382
4,504,"[19894, 2237, 6628, 49533, 5025, 15392, 19678,...",2201
24,629,"[34126, 30305, 11895, 46069, 38419, 16325, 591...",3351
36,832,"[13176, 31720, 47209, 21137, 11422, 19204, 241...",3683
46,3834,"[25146, 24363, 20282, 27413, 22159, 21497, 12341]",2371
53,4814,"[40092, 40824, 25551, 40146, 432, 4357, 26756,...",1423
85,6364,"[21463, 11574, 49215, 44032, 24195, 39409]",2822
91,6769,"[6846, 24852, 26384, 4210, 9515, 8859, 15125, ...",4201
106,7585,"[25495, 5013, 45443, 3522, 35131, 19535, 32732...",2059
114,8535,"[18135, 39032, 15404, 1541, 14634, 3298]",3366


In [113]:
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

t_last_order['f1']=0
t_last_order['apk']=0

for i in t_last_order.index:
    t_last_order['f1'][i]=f1(t_last_order['product_id_y'][i],t_last_order['product_id_x'][i])
    t_last_order['apk'][i]=apk(t_last_order['product_id_y'][i],t_last_order['product_id_x'][i])
    
print('f1 mean:',np.mean(t_last_order.f1))
print('apk mean:',np.mean(t_last_order.apk))

/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/grecia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


f1 mean: 0.01893439013650374
apk mean: 0.029502421840598855
